下載資料用，如果已經下載好可以不執行

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/dataset.zip"  # 你的 zip 路徑
extract_path = "/content/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

安裝函式庫

In [ ]:
!pip install torch torchvision torchaudio
!pip install pillow
!pip install pandas
!pip install tqdm

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from PIL import Image
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
from torchvision import models

處理Data

In [ ]:
# --- Custom Dataset for unlabeled/test ---
class ImageFolderWithoutLabels(Dataset):
    def __init__(self, root, transform=None, extensions=("jpg","jpeg","png")):
        self.root = root
        self.transform = transform
        self.extensions = extensions

        # 找所有符合副檔名的檔案
        self.images = [
            os.path.join(root, fname) for fname in os.listdir(root)
            if fname.lower().endswith(self.extensions)
        ]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_path  # 回傳圖片和路徑，方便之後保存結果

# --- 主函數 ---
def get_loaders(root="./data/data", batch_size=32, valid_ratio=0.1):

    train_dir = os.path.join(root, "train")
    test_dir  = os.path.join(root, "test")
    unlabeled_dir = os.path.join(root, "unlabeled")

    # ---------- Transforms ----------
    train_tf = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),  # 可留可不留
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
    ])

    eval_tf = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                             [0.229,0.224,0.225])
    ])

    # ---------- Train/Valid ----------
    full_train = datasets.ImageFolder(train_dir, transform=train_tf)
    total = len(full_train)
    valid_len = int(total * valid_ratio)
    train_len = total - valid_len
    train_ds, valid_ds = random_split(full_train, [train_len, valid_len])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

    # ---------- Test ----------
    test_ds = ImageFolderWithoutLabels(test_dir, transform=eval_tf)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    # ---------- Unlabeled ----------
    unlabeled_ds = ImageFolderWithoutLabels(unlabeled_dir, transform=eval_tf)
    unlabeled_loader = DataLoader(unlabeled_ds, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader, test_loader, unlabeled_loader


正式訓練
資料必須放在指定位置
    data_root = "your data root direct"
    save_path = "your model path"

In [ ]:
def main():
    # ------------------ Config ------------------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_epochs = 8
    batch_size = 32
    learning_rate = 0.01
    num_classes = 2
    data_root = "/content/data/data"
    save_path = "best_model_ef5.pth"

    # ------------------ Data ------------------
    train_loader, valid_loader, test_loader, _ = get_loaders(
        root=data_root,
        batch_size=batch_size
    )

    # ------------------ Model ------------------
    model = models.efficientnet_b7(
    weights=models.EfficientNet_B7_Weights.IMAGENET1K_V1
    )
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=learning_rate,
        momentum=0.9,
        weight_decay=1e-3
    )

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,
        eta_min=1e-4  # 最小 lr
    )

    best_acc = 0.0

    # model.load_state_dict(torch.load(save_path))
    # ------------------ Training ------------------
    for epoch in range(num_epochs):
        print("Model device:", next(model.parameters()).device)
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", ncols=100)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        avg_train_loss = running_loss / len(train_loader)
        scheduler.step()  # learning rate decay
        # ------------------ Validation ------------------
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        loop_val = tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]", ncols=100)
        with torch.no_grad():
            for images, labels in loop_val:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                loop_val.set_postfix(val_loss=loss.item())
        avg_val_loss = val_loss / len(valid_loader)
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {avg_train_loss:.4f} "
              f"Valid Loss: {avg_val_loss:.4f} "
              f"Valid Acc: {accuracy:.2f}%")

        if accuracy > best_acc:
            best_acc = accuracy
            torch.save(model.state_dict(), save_path)

    # ------------------ Test Prediction ------------------
    print("Loading best model for test prediction...")
    model.load_state_dict(torch.load(save_path))
    model.eval()
    preds_list = []
    image_paths = []

    with torch.no_grad():
        loop_test = tqdm(test_loader, desc="Predicting Test", ncols=100)
        for images, paths in loop_test:
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)[:,1]
            preds = (probs <= 0.5).int().cpu().tolist()
            preds_list.extend(preds)
            image_paths.extend(paths)

    # 去掉副檔名
    filenames = [os.path.splitext(os.path.basename(p))[0] for p in image_paths]
    submission = pd.DataFrame({"filename": filenames, "label": preds_list})
    submission.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

# ------------------ Entry Point ------------------
if __name__ == "__main__":
    main()
